In [136]:
def T(formula):
    """
    Transforms a given formula according to the specified rules and removes double negations.
    """

    # Recursive transformation
    def transform(formula):
        # Base case: If the formula is a single proposition
        if formula.isalpha():
            return formula

        # Recursive transformation for negation
        if formula.startswith('¬'):
            subformula = transform(formula[1:])
            # Remove double negation
            if subformula.startswith('¬'):
                return subformula[1:]
            else:
                return '¬' + subformula

        # Recursive transformation for conjunction, disjunction, and implication
        depth = 0
        for i, char in enumerate(formula):
            if char == '(':
                depth += 1
            elif char == ')':
                depth -= 1
            elif depth == 0:
                if char == '∧':
                    left = formula[:i].strip()
                    right = formula[i+1:].strip()
                    return transform(left) + ' ∧ ' + transform(right)
                elif char == '∨':
                    left = formula[:i].strip()
                    right = formula[i+1:].strip()
                    return '¬(¬' + transform(left) + ' ∧ ¬' + transform(right) + ')'
                elif char == '→':
                    left = formula[:i].strip()
                    right = formula[i+1:].strip()
                    return '¬(' + transform(left) + ' ∧ ¬' + transform(right) + ')'

        # If the formula is enclosed in parentheses, remove them and transform
        if formula.startswith('(') and formula.endswith(')'):
            return transform(formula[1:-1])

        return formula

    # Start the transformation process
    return transform(formula)

In [137]:
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

def parse_formula(formula):
    """
    Parses the formula and returns the root of the parse tree.
    """

    # Helper function to find the main operator and its index
    def find_main_operator(formula):
        level = 0
        for i, char in enumerate(formula):
            if char == '(':
                level += 1
            elif char == ')':
                level -= 1
            elif level == 0 and char in ['∧', '∨', '→']:
                return char, i
        return None, -1

    formula = formula.strip()
    while formula.startswith('(') and formula.endswith(')'):
        formula = formula[1:-1].strip()

    operator, index = find_main_operator(formula)
    if operator:
        node = Node(operator)
        node.left = parse_formula(formula[:index].strip())
        node.right = parse_formula(formula[index + 1:].strip())
        return node
    elif formula.startswith('¬'):
        node = Node('¬')
        node.right = parse_formula(formula[1:].strip())
        return node
    else:
        return Node(formula)

def parse_to_dag(node, seen_subtrees):
    """
    Transforms the parse tree into a DAG by merging common subtrees.
    """
    if node is None:
        return None

    # Create a unique representation of the current subtree
    subtree_repr = (node.value, id(node.left), id(node.right))

    # Check if the subtree has already been seen
    if subtree_repr in seen_subtrees:
        # Reuse the existing node
        return seen_subtrees[subtree_repr]

    # This is a new subtree, add it to the seen subtrees
    seen_subtrees[subtree_repr] = node

    # Recursively process the left and right children
    node.left = parse_to_dag(node.left, seen_subtrees)
    node.right = parse_to_dag(node.right, seen_subtrees)

    return node

In [138]:
def print_tree(node, depth=0, prefix="Root: "):
    """
    Prints the parse tree with appropriate indentation.
    """
    if node is not None:
        print_tree(node.right, depth + 1, "R----")
        print("   " * depth + prefix + node.value)
        print_tree(node.left, depth + 1, "L----")

def print_dag(node, depth=0, seen_nodes=None, node_ids=None):
    """
    Prints the DAG with appropriate indentation, showing connections.
    """
    if node is None:
        return

    if seen_nodes is None:
        seen_nodes = set()
        node_ids = {}

    # Assign a unique identifier to the node if it's not already seen
    if node not in node_ids:
        node_ids[node] = len(node_ids) + 1  # IDs are 1-indexed

    node_id = node_ids[node]

    # Print node value and indicate if it's a shared node
    shared = ' (shared)' if node in seen_nodes else ''
    print(f"{'   ' * depth}Node {node_id}: {node.value}{shared}")

    # If this node has already been printed, it's shared, and we stop printing its children
    if node in seen_nodes:
        return

    seen_nodes.add(node)

    # Recursively print left and right children
    if node.left:
        print_dag(node.left, depth + 1, seen_nodes, node_ids)
    if node.right:
        print_dag(node.right, depth + 1, seen_nodes, node_ids)


In [139]:
def apply_force_laws(node, valuation, force_laws_applied):
    if node is None or node in force_laws_applied:
        return True

    force_laws_applied.add(node)
    value = valuation.get(node, None)

    if node.value == '∧':
        left_val = valuation.get(node.left)
        right_val = valuation.get(node.right)
        if value is True:
            valuation[node.left] = True
            valuation[node.right] = True
        elif left_val is False or right_val is False:
            valuation[node] = False
        elif left_val is True and right_val is True:
            valuation[node] = True

    elif node.value == '¬':
        child_val = valuation.get(node.right)
        if value is not None:
            valuation[node.right] = not value
        if child_val is not None:
            valuation[node] = not child_val

    left_satisfied = apply_force_laws(node.left, valuation, force_laws_applied)
    right_satisfied = apply_force_laws(node.right, valuation, force_laws_applied)

    return left_satisfied and right_satisfied

def is_satisfiable(root):
    valuation = {root: True}
    force_laws_applied = set()

    satisfiable = apply_force_laws(root, valuation, force_laws_applied)

    # Post-processing: Check consistency from bottom-up
    for node in force_laws_applied:
        if node.value == '∧':
            if valuation.get(node.left, False) and valuation.get(node.right, False):
                valuation[node] = True
            else:
                valuation[node] = False

    return satisfiable and valuation.get(root, False)

In [140]:
# Example usage
input_formula = "p ∧ ¬(q ∨ ¬p)"
translated_formula = T(input_formula)
print("Original formula:", input_formula)
print("Translated formula:", translated_formula)

Original formula: p ∧ ¬(q ∨ ¬p)
Translated formula: p ∧ (¬q ∧ ¬¬p)


In [141]:
# Example usage
input_formula = "p ∧ ¬¬(¬q ∧ ¬¬p)"
root = parse_formula(input_formula)
print("Parse tree of the formula:")
print_tree(root)

Parse tree of the formula:
                  R----p
               R----¬
            R----¬
         R----∧
               R----q
            L----¬
      R----¬
   R----¬
Root: ∧
   L----p


In [142]:
# Example usage remains the same

# Transform parse tree to DAG
seen_subtrees = {}
root = parse_to_dag(root, seen_subtrees)

print("DAG of the formula:")
print_dag(root)

DAG of the formula:
Node 1: ∧
   Node 2: p
   Node 3: ¬
      Node 4: ¬
         Node 5: ∧
            Node 6: ¬
               Node 7: q
            Node 8: ¬
               Node 9: ¬
                  Node 2: p (shared)


In [143]:
# Usage (you need to construct the DAG from your formula first)
seen_subtrees = {}
root = parse_to_dag(root, seen_subtrees)

satisfiable = is_satisfiable(root)
print("The formula is satisfiable." if satisfiable else "The formula is not satisfiable.")

The formula is satisfiable.


In [144]:
# Example usage
input_formula = "p ∨ q"
translated_formula = T(input_formula)
print("Original formula:", input_formula)
print("Translated formula:", translated_formula)

Original formula: p ∨ q
Translated formula: ¬(¬p ∧ ¬q)


In [145]:
# Example usage
input_formula = "¬(¬p ∧ ¬q)"
root = parse_formula(input_formula)
print("Parse tree of the formula:")
print_tree(root)

Parse tree of the formula:
         R----q
      R----¬
   R----∧
         R----p
      L----¬
Root: ¬


In [146]:
# Example usage remains the same

# Transform parse tree to DAG
seen_subtrees = {}
root = parse_to_dag(root, seen_subtrees)

print("DAG of the formula:")
print_dag(root)

DAG of the formula:
Node 1: ¬
   Node 2: ∧
      Node 3: ¬
         Node 4: p
      Node 5: ¬
         Node 6: q


In [147]:
# Usage (you need to construct the DAG from your formula first)
seen_subtrees = {}
root = parse_to_dag(root, seen_subtrees)

satisfiable = is_satisfiable(root)
print("The formula is satisfiable." if satisfiable else "The formula is not satisfiable.")

The formula is satisfiable.


In [148]:
# Example usage
input_formula = "(p) ∨ (p ∨ p)"
translated_formula = T(input_formula)
print("Original formula:", input_formula)
print("Translated formula:", translated_formula)

Original formula: (p) ∨ (p ∨ p)
Translated formula: ¬(¬p ∧ ¬¬(¬p ∧ ¬p))


In [149]:
# Example usage
input_formula = "¬(¬p ∧ ¬¬(¬p ∧ ¬p))"
root = parse_formula(input_formula)
print("Parse tree of the formula:")
print_tree(root)

Parse tree of the formula:
                  R----p
               R----¬
            R----∧
                  R----p
               L----¬
         R----¬
      R----¬
   R----∧
         R----p
      L----¬
Root: ¬


In [150]:
# Example usage remains the same

# Transform parse tree to DAG
seen_subtrees = {}
root = parse_to_dag(root, seen_subtrees)

print("DAG of the formula:")
print_dag(root)

DAG of the formula:
Node 1: ¬
   Node 2: ∧
      Node 3: ¬
         Node 4: p
      Node 5: ¬
         Node 6: ¬
            Node 7: ∧
               Node 8: ¬
                  Node 4: p (shared)
               Node 9: ¬
                  Node 4: p (shared)


In [151]:
# Usage (you need to construct the DAG from your formula first)
seen_subtrees = {}
root = parse_to_dag(root, seen_subtrees)

satisfiable = is_satisfiable(root)
print("The formula is satisfiable." if satisfiable else "The formula is not satisfiable.")

The formula is satisfiable.


In [152]:
# Example usage
input_formula = "¬p ∧ (q ∨ r)"
translated_formula = T(input_formula)
print("Original formula:", input_formula)
print("Translated formula:", translated_formula)

Original formula: ¬p ∧ (q ∨ r)
Translated formula: ¬p ∧ ¬(¬q ∧ ¬r)


In [153]:
# Example usage
input_formula = "¬(p ∧ ¬(¬q ∧ ¬r))"
root = parse_formula(input_formula)
print("Parse tree of the formula:")
print_tree(root)

Parse tree of the formula:
               R----r
            R----¬
         R----∧
               R----q
            L----¬
      R----¬
   R----∧
      L----p
Root: ¬


In [154]:
# Example usage remains the same

# Transform parse tree to DAG
seen_subtrees = {}
root = parse_to_dag(root, seen_subtrees)

print("DAG of the formula:")
print_dag(root)

DAG of the formula:
Node 1: ¬
   Node 2: ∧
      Node 3: p
      Node 4: ¬
         Node 5: ∧
            Node 6: ¬
               Node 7: q
            Node 8: ¬
               Node 9: r


In [155]:
# Usage (you need to construct the DAG from your formula first)
seen_subtrees = {}
root = parse_to_dag(root, seen_subtrees)

satisfiable = is_satisfiable(root)
print("The formula is satisfiable." if satisfiable else "The formula is not satisfiable.")

The formula is satisfiable.


In [156]:
# Example usage
input_formula = "(p ∧ q) ∨ r"
translated_formula = T(input_formula)
print("Original formula:", input_formula)
print("Translated formula:", translated_formula)

Original formula: (p ∧ q) ∨ r
Translated formula: ¬(¬p ∧ q ∧ ¬r)


In [157]:
# Example usage
input_formula = "¬(¬(p ∧ q) ∧ ¬r)"
root = parse_formula(input_formula)
print("Parse tree of the formula:")
print_tree(root)

Parse tree of the formula:
         R----r
      R----¬
   R----∧
            R----q
         R----∧
            L----p
      L----¬
Root: ¬


In [158]:
# Example usage remains the same

# Transform parse tree to DAG
seen_subtrees = {}
root = parse_to_dag(root, seen_subtrees)

print("DAG of the formula:")
print_dag(root)

DAG of the formula:
Node 1: ¬
   Node 2: ∧
      Node 3: ¬
         Node 4: ∧
            Node 5: p
            Node 6: q
      Node 7: ¬
         Node 8: r


In [159]:
# Usage (you need to construct the DAG from your formula first)
seen_subtrees = {}
root = parse_to_dag(root, seen_subtrees)

satisfiable = is_satisfiable(root)
print("The formula is satisfiable." if satisfiable else "The formula is not satisfiable.")

The formula is satisfiable.


In [160]:
# Example usage
input_formula = "p ∧ ¬p"
translated_formula = T(input_formula)
print("Original formula:", input_formula)
print("Translated formula:", translated_formula)

Original formula: p ∧ ¬p
Translated formula: p ∧ ¬p


In [161]:
# Example usage
input_formula = "p ∧ ¬p"
root = parse_formula(input_formula)
print("Parse tree of the formula:")
print_tree(root)

Parse tree of the formula:
      R----p
   R----¬
Root: ∧
   L----p


In [162]:
# Example usage remains the same

# Transform parse tree to DAG
seen_subtrees = {}
root = parse_to_dag(root, seen_subtrees)

print("DAG of the formula:")
print_dag(root)

DAG of the formula:
Node 1: ∧
   Node 2: p
   Node 3: ¬
      Node 2: p (shared)


In [163]:
# Usage (you need to construct the DAG from your formula first)
seen_subtrees = {}
root = parse_to_dag(root, seen_subtrees)

satisfiable = is_satisfiable(root)
print("The formula is satisfiable." if satisfiable else "The formula is not satisfiable.")

The formula is not satisfiable.
